In [ ]:
#!pip install pycrypto
#!pip uninstall pycrypto
# !pip install pycryptodome #==3.10.1

# !pip3 install --upgrade cryptography

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 7.5 MB/s eta 0:00:00


In [26]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes, serialization
from cryptography.hazmat.primitives.asymmetric import rsa, padding, utils
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives.padding import PKCS7
import hmac, hashlib
import base64
import os
import rsa

In [24]:
# from cryptography.hazmat.primitives import padding
# from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
# from cryptography.hazmat.backends import default_backend
# import base64
# import os

In [27]:
# AES Encryption/Decryption
def generate_random_key(key_length):
    return os.urandom(key_length // 8)

def pad_message(message):
    padder = padding.PKCS7(128).padder()
    padded_message = padder.update(message) + padder.finalize()
    return padded_message

def unpad_message(padded_message):
    unpadder = padding.PKCS7(128).unpadder()
    message = unpadder.update(padded_message) + unpadder.finalize()
    return message

def aes_encrypt(message, key, mode):
    backend = default_backend()
    iv = b'\x00' * 16  # Initialization vector (for CBC mode)

    if mode == 'ECB':
        cipher = Cipher(algorithms.AES(key), modes.ECB(), backend=backend)
    elif mode == 'CBC':
        cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=backend)
    else:
        raise ValueError("Invalid mode")

    encryptor = cipher.encryptor()
    padded_message = pad_message(message)
    ciphertext = encryptor.update(padded_message) + encryptor.finalize()
    return ciphertext

def aes_decrypt(ciphertext, key, mode):
    backend = default_backend()
    iv = b'\x00' * 16  # Initialization vector (for CBC mode)

    if mode == 'ECB':
        cipher = Cipher(algorithms.AES(key), modes.ECB(), backend=backend)
    elif mode == 'CBC':
        cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=backend)
    else:
        raise ValueError("Invalid mode")

    decryptor = cipher.decryptor()
    decrypted_message = decryptor.update(ciphertext) + decryptor.finalize()
    unpadded_message = unpad_message(decrypted_message)
    return unpadded_message


# RSA Encryption/Decryption
def rsa_encrypt_decrypt(operation, text, key):
    if operation == 'Encrypt':
        key = rsa.PublicKey.load_pkcs1(key)
        encrypted = rsa.encrypt(text.encode(), key)
        return base64.urlsafe_b64encode(encrypted).decode()
    elif operation == 'Decrypt':
        key = rsa.PrivateKey.load_pkcs1(key)
        decrypted = rsa.decrypt(base64.urlsafe_b64decode(text), key).decode()
        return decrypted

# Hashing
def generate_hash(data, hash_mode):
    if hash_mode == 'SHA1':
        hash_algorithm = hashlib.sha1()
    elif hash_mode == 'SHA256':
        hash_algorithm = hashlib.sha256()
    elif hash_mode == 'MD5':
        hash_algorithm = hashlib.md5()
    else:
        raise ValueError("Invalid hash mode")
    # Update the hash algorithm with the encoded data
    hash_algorithm.update(data.encode())
    # Return the hexadecimal representation of the hash
    return hash_algorithm.hexdigest()

# Digital Signature using RSA
def generate_signature(message, private_key):
    signature = rsa.sign(message, private_key, 'SHA-256')
    return signature

def verify_signature(message, signature, public_key):
    try:
        rsa.verify(message, signature, public_key)
        return True
    except rsa.pkcs1.VerificationError:
        return False

# MAC Generation
def generate_mac(message, algorithm_name):
    algorithm = hashlib.new(algorithm_name)
    algorithm.update(message.encode())
    mac = algorithm.digest()
    return mac.hex()

In [30]:
# Main program
if __name__ == '__main__':
    (publicKey, privateKey) = rsa.newkeys(512)
    while True:
        print("1. AES Encryption/Decryption")
        print("2. RSA Encryption/Decryption")
        print("3. Hashing")
        print("4. Digital Signature using RSA")
        print("5. MAC Generation")
        print("6. Exit")

        choice = input("Select an option: ")


        if choice == '1':
            print("AES Encryption and Decryption")

            mode = input("Enter mode (ECB/CBC): ").upper()  # Convert input to uppercase
            key_length = int(input("Enter key length (128/192/256 bits): "))
            if key_length not in [128, 192, 256]:
                print("Invalid key length. Please choose 128, 192, or 256 bits.")
                #return

            key = generate_random_key(key_length)  # Generate key for both encryption and decryption

            while True:
                print("\n1. Encrypt")
                print("2. Decrypt")
                print("3. Exit")
                choice = input("Select an option: ")

                if choice == '1':
                    plaintext = input("Enter plaintext: ").encode()
                    ciphertext = aes_encrypt(plaintext, key, mode)
                    print("Ciphertext:", base64.b64encode(ciphertext).decode())
                    print(f"Generated {key_length}-bit key:", key.hex())

                elif choice == '2':
                    ciphertext = base64.b64decode(input("Enter ciphertext: "))
                    decrypted_message = aes_decrypt(ciphertext, key, mode)
                    print("Decrypted message:", decrypted_message.decode())

                elif choice == '3':
                    print("Exiting the program.")
                    break

                else:
                    print("Invalid choice. Please select a valid option.")

        elif choice == '2':
            #(publicKey, privateKey) = rsa.newkeys(512)
            operation = input("Enter operation (Encrypt/Decrypt): ")
            text = input("Enter plaintext/ciphertext: ")
            if operation == 'Encrypt':
                key = publicKey.save_pkcs1().decode()
            elif operation == 'Decrypt':
                key = privateKey.save_pkcs1().decode()
            result = rsa_encrypt_decrypt(operation, text, key)
            print("Result:", result)

        elif choice == '3':
            data = input("Enter data to be hashed: ")
            hash_mode = input("Enter hash mode (SHA1/SHA256/MD5): ")
            hash_value = generate_hash(data, hash_mode)
            print("Generated Hash:", hash_value)


        elif choice == '4':
            print("RSA Digital Signature Generation and Verification")

            # Generate RSA key pair
            (pubkey, privkey) = rsa.newkeys(2048)
            print("RSA Key Pair Generated")

            while True:
                print("\n1. Generate Signature")
                print("2. Verify Signature")
                print("3. Exit")
                choice = input("Select an option: ")

                if choice == '1':
                    message = input("Enter the message to sign: ").encode()
                    signature = generate_signature(message, privkey)
                    print("Generated Signature:", signature.hex())

                elif choice == '2':
                    message = input("Enter the original message: ").encode()
                    signature_hex = input("Enter the signature in hexadecimal format: ")
                    signature = bytes.fromhex(signature_hex)

                    is_verified = verify_signature(message, signature, pubkey)
                    if is_verified:
                        print("Signature is valid. Message integrity and authenticity are confirmed.")
                    else:
                        print("Signature is invalid. Message may have been tampered with.")

                elif choice == '3':
                    print("Exiting the program.")
                    break

                else:
                    print("Invalid choice. Please select a valid option.")

        elif choice == '5':
              message = input("Enter message for MAC generation: ")
              print("Available MAC algorithms:")
              print("1. sha256")
              print("2. sha1")
              algo_choice = input("Select a MAC algorithm (1/2): ")

              if algo_choice == '1':
                  algorithm_name = 'sha256'
              elif algo_choice == '2':
                  algorithm_name = 'sha1'
              else:
                  print("Invalid MAC algorithm choice.")
                  continue

              mac = generate_mac(message, algorithm_name)
              print("MAC Value:", mac)

        elif choice == '6':
            print("Exiting...")
            break

        else:
            print("Invalid choice. Please select a valid option.")

1. AES Encryption/Decryption
2. RSA Encryption/Decryption
3. Hashing
4. Digital Signature using RSA
5. MAC Generation
6. Exit
Select an option: 2
Enter operation (Encrypt/Decrypt): Encrypt
Enter plaintext/ciphertext: asdjkaskd
Result: Q-CPeIUl6RuF2BDwvCGaoP_w8AHRmPNOmG5ejrtOxJWvC4L60CzoGT1fwR43AS4Z8OhyUg7JO7Vp7yG0ZbP6Gw==
1. AES Encryption/Decryption
2. RSA Encryption/Decryption
3. Hashing
4. Digital Signature using RSA
5. MAC Generation
6. Exit
Select an option: 2
Enter operation (Encrypt/Decrypt): Decrypt
Enter plaintext/ciphertext: Q-CPeIUl6RuF2BDwvCGaoP_w8AHRmPNOmG5ejrtOxJWvC4L60CzoGT1fwR43AS4Z8OhyUg7JO7Vp7yG0ZbP6Gw==
Result: asdjkaskd
1. AES Encryption/Decryption
2. RSA Encryption/Decryption
3. Hashing
4. Digital Signature using RSA
5. MAC Generation
6. Exit
Select an option: 3
Enter data to be hashed: sadasd
Enter hash mode (SHA1/SHA256/MD5): MD5
Generated Hash: c99a11a53a3748269e3f86d7ac38df11
1. AES Encryption/Decryption
2. RSA Encryption/Decryption
3. Hashing
4. Digital Signat